<div class='status' style='background-color: #183a4b; color: white; padding-top: 4px; padding-bottom: 4px; padding-left: 20px; padding-right: 20px; border-radius: 10px; font-family: Arial, sans-serif; font-size: 26px; display: inline-block; text-align: center; box-shadow: 0px 3px 4px rgba(0, 0, 0, 0.5);'><b>AI MedReview</B> - Fake GP Surgery Reviews | Processing Pipeline</div>

## Loading Libraries

In [8]:
# Importing default Libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import datetime
import os
from tqdm import tqdm

pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

# Hi-resolution Plots and Matplotlib inline
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# Set the maximum number of rows and columns to be displayed
warnings.filterwarnings("ignore")

# "magic commands" to enable autoreload of your imported packages
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**Loading Libraries** from AI MedReview Pipeline

In [9]:
from ai_medreview.fake_data_pipeline import *

<div class='status' style='background-color: #f5c244; color: white; padding-top: 2px; padding-bottom: 2px; padding-left: 7px; padding-right: 7px; border-radius: 6px; font-family: Arial, sans-serif; font-size: 18px; display: inline-block; text-align: center; box-shadow: 0px 3px 4px rgba(0, 0, 0, 0.2);'><b>Loading</b> dataset</div>

Loading 1/3 datasets chunks for processing

In [10]:
fake_data = pd.read_csv("../ai_medreview/data/fake_gp_reviews_llm_01.csv")
fake_data.columns = ["free_text", "generation_category"]
fake_data.head(2)

,free_text,generation_category
0,The staff were very helpful and kind.\n\nI rec...,Patient Respect
1,The staff were friendly but forgot my appointm...,Patient Respect


In [11]:
fake_data = word_count(fake_data)

2025-01-30 19:04:28.781 | INFO     | ai_medreview.utils:wrapper:15 - 🖥️    Started: 'word_count'
2025-01-30 19:04:28.808 | INFO     | ai_medreview.utils:wrapper:21 - ✅ Completed: 'word_count' ⚡️0.024931 sec


In [12]:
fake_data = sentiment_analysis(fake_data, "free_text")

2025-01-30 19:04:29.271 | INFO     | ai_medreview.fake_data_pipeline:sentiment_analysis:77 - 💛 Sentiment Analysis - Functions started.
█████████████████████████████████████| 1935/1935 [01:41<00:00, 19.01it/s]


In [13]:
fake_data = cleanup_neutral_sentiment(fake_data, "free_text")

2025-01-30 19:06:11.110 | INFO     | ai_medreview.fake_data_pipeline:cleanup_neutral_sentiment:162 - 🧻 Cleanup_neutral_sentiment - if free_text and do_better isna()


In [14]:
fake_data = feedback_classification(fake_data, batch_size=8)

2025-01-30 19:06:11.153 | INFO     | ai_medreview.utils:wrapper:15 - 🖥️    Started: 'feedback_classification'
Processing batches: 100%|██████████████████████████████████████| 242/242 [4:59:43<00:00, 74.31s/it]
2025-01-31 00:05:58.634 | INFO     | ai_medreview.utils:wrapper:21 - ✅ Completed: 'feedback_classification' ⚡️17987.476952 sec


In [15]:
fake_data.head(2)

,free_text,generation_category,free_text_len,sentiment_free_text,sentiment_score_free_text,feedback_labels
0,The staff were very helpful and kind.\n\nI rec...,Patient Respect,246,positive,0.959466,Reception Staff Interaction
1,The staff were friendly but forgot my appointm...,Patient Respect,933,negative,0.878586,Communication Effectiveness


In [ ]:
fake_data = emotion_classification(fake_data, "free_text", classifier=classifier)

2025-01-31 00:05:58.922 | INFO     | ai_medreview.utils:wrapper:15 - 🖥️    Started: 'emotion_classification'
██████████▍                             | 504/1935 [01:12<03:03,  7.79it/s]

In [ ]:
fake_data.head(2)

In [ ]:
fake_data.to_csv('../ai_medreview/data/fake_gp_reviews_01_processed.csv', index=False)